In [49]:
from keras.preprocessing.image import ImageDataGenerator

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

In [33]:
import os

  
os.chdir(r'C:\Users\MARG\Documents\radar_classification\train\walking')
print(os.getcwd())
COUNT = 1
  
# Function to increment count 
# to make the files sorted.
def increment():
    global COUNT
    COUNT = COUNT + 1
  
  
for f in os.listdir():
    f_name, f_ext = os.path.splitext(f)
    f_name = "walking" + str(COUNT)
    increment()
  
    new_name = '{}{}'.format(f_name, f_ext)
    os.rename(f, new_name)

C:\Users\MARG\Documents\radar_classification\train\walking


FileExistsError: [WinError 183] Cannot create a file when that file already exists: 'walking10.png' -> 'walking2.png'

In [60]:
import numpy as np
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img

datagen = ImageDataGenerator(
        rotation_range=40,
        width_shift_range=0.2,
        height_shift_range=0.2,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True,
        fill_mode='nearest')

img = load_img(r'C:\Users\MARG\Documents\radar_classification\train\walking\walking1.png')  # this is a PIL image
x = img_to_array(img)  # this is a Numpy array with shape (3, 150, 150)
print(x.shape)
x = np.reshape(x,(1475,804,3))
print(x.shape)
x = x.reshape((1,) + x.shape)
print(x.shape)# this is a Numpy array with shape (1, 3, 150, 150)

# the .flow() command below generates batches of randomly transformed images
# and saves the results to the `preview/` directory
i = 0
for batch in datagen.flow(x, batch_size=1,
                          save_to_dir=r'C:\Users\MARG\Documents\radar_classification\train\preview', save_prefix='walking', save_format='jpeg'):
    i += 1
    if i > 20:
        break  # otherwise the generator would loop indefinitely

(804, 1475, 3)
(1475, 804, 3)
(1, 1475, 804, 3)


In [61]:
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D
from keras.layers import Activation, Dropout, Flatten, Dense

model = Sequential()
model.add(Conv2D(32, (3, 3), input_shape=(1475, 804, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(32, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

model.add(Conv2D(64, (3, 3)))
model.add(Activation('relu'))
model.add(MaxPooling2D(pool_size=(2, 2)))

In [62]:
model.add(Flatten())  # this converts our 3D feature maps to 1D feature vectors
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(loss='binary_crossentropy',
              optimizer='rmsprop',
              metrics=['accuracy'])

In [66]:
batch_size = 16

# this is the augmentation configuration we will use for training
train_datagen = ImageDataGenerator(
        rescale=1./255,
        shear_range=0.2,
        zoom_range=0.2,
        horizontal_flip=True)

# this is the augmentation configuration we will use for testing:
# only rescaling
test_datagen = ImageDataGenerator(rescale=1./255)

# this is a generator that will read pictures found in
# subfolers of 'data/train', and indefinitely generate
# batches of augmented image data
train_generator = train_datagen.flow_from_directory(
        r'C:\Users\MARG\Documents\radar_classification\train',  # this is the target directory
        target_size=(, 150),  # all images will be resized to 150x150
        batch_size=batch_size,
        class_mode='binary')  # since we use binary_crossentropy loss, we need binary labels
print(train_generator)
# this is a similar generator, for validation data
validation_generator = test_datagen.flow_from_directory(
        r'C:\Users\MARG\Documents\radar_classification\train',
        target_size=(150, 150),
        batch_size=batch_size,
        class_mode='binary')

Found 345 images belonging to 3 classes.
Found 345 images belonging to 3 classes.


In [64]:
model.fit_generator(
        train_generator,
        steps_per_epoch=500 // batch_size,
        epochs=50,
        validation_data=validation_generator,
        validation_steps=57 // batch_size)
model.save_weights('first_try.h5')  # always save your weights after training or during training

Epoch 1/50


InvalidArgumentError:  Input to reshape is a tensor with 295936 values, but the requested shape requires a multiple of 1141504
	 [[node sequential_9/flatten_4/Reshape (defined at C:\Users\MARG\Anaconda3\envs\radar_classifcation\lib\site-packages\keras\layers\core.py:672) ]] [Op:__inference_train_function_5789]

Errors may have originated from an input operation.
Input Source operations connected to node sequential_9/flatten_4/Reshape:
 sequential_9/flatten_4/Const (defined at C:\Users\MARG\Anaconda3\envs\radar_classifcation\lib\site-packages\keras\layers\core.py:667)	
 sequential_9/max_pooling2d_18/MaxPool (defined at C:\Users\MARG\Anaconda3\envs\radar_classifcation\lib\site-packages\keras\layers\pooling.py:355)

Function call stack:
train_function
